In [1]:
from typing import Dict
from collections.abc import Iterable

import numpy as np
import pandas as pd
from statsmodels.stats.multitest import multipletests
from tqdm.auto import tqdm

from pyplier.AUC import AUC
from pyplier.copyMat import copyMat
from pyplier.PLIERRes import PLIERResults
from pathlib import Path

In [2]:
data_dir = Path("/workspaces/pyplier/tests/data/crossVal")
plierRes = PLIERResults.from_disk(data_dir / "plierRes.json.gz")

priorMat = pd.read_csv(data_dir / "priorMat.csv.gz", index_col="gene")
priorMatcv = pd.read_csv(data_dir / "priorMatcv.csv.gz", index_col=0)

In [3]:
out = pd.DataFrame(
    data=np.empty(shape=(0, 4)), columns=["pathway", "LV index", "AUC", "p-value"]
)
out_dict = dict()
ii = plierRes.U.loc[:, plierRes.U.sum(axis=0) > 0].columns
Uauc = pd.DataFrame(
    np.zeros(shape=plierRes.U.shape), index=plierRes.U.index, columns=plierRes.U.columns
)
Up = pd.DataFrame(
    np.ones(shape=plierRes.U.shape), index=plierRes.U.index, columns=plierRes.U.columns
)

In [4]:
ii

Index(['LV1', 'LV2', 'LV3', 'LV4', 'LV5', 'LV6', 'LV7', 'LV8', 'LV9', 'LV10',
       'LV11', 'LV14', 'LV15', 'LV18', 'LV20', 'LV23', 'LV24', 'LV26', 'LV27',
       'LV29', 'LV30'],
      dtype='object')

In [5]:
i = "LV1"
iipath = plierRes.U.loc[(plierRes.U.loc[:, i] > 0), i].index

In [6]:
plierRes.U["LV1"][plierRes.U["LV1"] > 0]

REACTOME_GENERIC_TRANSCRIPTION_PATHWAY    0.03892
Name: LV1, dtype: float64

In [7]:
iipath

Index(['REACTOME_GENERIC_TRANSCRIPTION_PATHWAY'], dtype='object')

In [8]:
iipath[0]

'REACTOME_GENERIC_TRANSCRIPTION_PATHWAY'

In [9]:
# for j in tqdm(iipath):
j = iipath[0]
iiheldout = (
    pd.concat([priorMat.loc[:, j], priorMatcv.loc[:, j]], axis=1)
    .apply(
        lambda x: True
        if (x[0] == 0) or ((x[0] > 0) and (x[1] == 0))
        else np.nan,  # use np.nan instead of False so that we can drop entries in the chain
        axis=1,
    )
    .dropna()
    .index
)

In [10]:
iiheldout

Index(['GAS6', 'MMP14', 'MARCKSL1', 'SPARC', 'CTSD', 'EPAS1', 'PALLD', 'PHC2',
       'LGALS3BP', 'SERPING1',
       ...
       'LDHA', 'LDHB', 'ACAP1', 'ACAP2', 'ACAP3', 'CFL2', 'CFL1', 'SELL',
       'GNGT2', 'SERPINH1'],
      dtype='object', name='gene', length=5672)

In [11]:
priorMat.loc[iiheldout, j]

gene
GAS6        0
MMP14       0
MARCKSL1    0
SPARC       0
CTSD        0
           ..
CFL2        0
CFL1        0
SELL        0
GNGT2       0
SERPINH1    0
Name: REACTOME_GENERIC_TRANSCRIPTION_PATHWAY, Length: 5672, dtype: int64

In [12]:
plierRes.Z.loc[iiheldout, i]

gene
GAS6        0.000000
MMP14       0.007516
MARCKSL1    0.000000
SPARC       0.000000
CTSD        0.000000
              ...   
CFL2        0.064740
CFL1        0.000000
SELL        0.009852
GNGT2       0.000000
SERPINH1    0.000000
Name: LV1, Length: 5672, dtype: float64

In [13]:
iiheldout

Index(['GAS6', 'MMP14', 'MARCKSL1', 'SPARC', 'CTSD', 'EPAS1', 'PALLD', 'PHC2',
       'LGALS3BP', 'SERPING1',
       ...
       'LDHA', 'LDHB', 'ACAP1', 'ACAP2', 'ACAP3', 'CFL2', 'CFL1', 'SELL',
       'GNGT2', 'SERPINH1'],
      dtype='object', name='gene', length=5672)

In [14]:
aucres = AUC(priorMat.loc[iiheldout, j], plierRes.Z.loc[iiheldout, i])

In [16]:
len(priorMat.loc[iiheldout, j][priorMat.loc[iiheldout, j] != 0])

54

In [17]:
aucres

{'low': 0.0071498315104708615, 'high': inf, 'auc': 0.6676489590337935, 'pval': 1.417381627952082e-06}

In [18]:
labels = priorMat.loc[iiheldout, j]
values = plierRes.Z.loc[iiheldout, i]

In [19]:
posii = labels[labels > 0]
negii = labels[labels <= 0]
posn = len(posii)
negn = len(negii)
posval = values[posii.index]
negval = values[negii.index]

In [20]:
posval

gene
SMARCD3    0.000000
NOTCH4     0.000000
ZNF708     0.122555
SP1        0.110843
SMAD4      0.093779
SMAD3      0.014073
ZNF3       0.000000
ZKSCAN1    0.115999
ZNF354B    0.064277
ZNF583     0.038242
TBL1X      0.000000
ZNF615     0.110649
NCOA3      0.141449
NCOA1      0.118420
ZNF493     0.114002
ZNF211     0.150148
ZNF514     0.200260
ZNF26      0.092609
ZNF25      0.146562
ZNF235     0.000000
ESR2       0.042226
MAML2      0.161881
MAML3      0.091114
TEAD2      0.000000
ZNF45      0.125577
ZNF43      0.064189
ZNF597     0.145484
ZNF599     0.000000
ZFP90      0.100559
CCNC       0.044585
ZNF141     0.111004
ZNF420     0.000000
TRIM33     0.199494
MYC        0.000000
ZNF641     0.086640
MAMLD1     0.015179
ZFP37      0.073333
ZNF446     0.000000
ZNF440     0.291950
SNW1       0.000000
MED1       0.179196
MED7       0.181450
ZNF627     0.007200
ZNF485     0.000000
NR2F6      0.000000
ZNF567     0.071099
ZNF169     0.072429
NR4A2      0.148702
NR4A1      0.000000
MED27      0.01

In [21]:
from scipy.stats import mannwhitneyu, norm

In [22]:
statistic, pvalue = mannwhitneyu(posval, negval, alternative="greater")

In [23]:
statistic

202546.0

In [24]:
pvalue

1.417381627952082e-06

In [25]:
x = posval
y = negval

In [26]:
n = len(x)
m = len(y)

In [27]:
alpha = 0.05

In [28]:
N = norm.ppf(1 - alpha / 2)

In [29]:
diffs = sorted([i - j for i in x for j in y])

In [30]:
# For an approximate 100(1-a)% confidence interval first calculate K:
nm = n * m
top = nm * (n + m + 1)
right = N * np.sqrt(top / 12)
left = (n * m) / 2
K = left - right

In [31]:
# The Kth smallest to the Kth largest of the n x m differences
# lx and ly should be > ~20
conf_int_low = diffs[round(K)]
conf_int_high = diffs[len(diffs) - round(K)]

In [32]:
from statsmodels.stats.nonparametric import rank_compare_2indep

In [93]:
from scipy.stats.contingency import relative_risk

In [34]:
posval.to_csv(
    "/workspaces/pyplier/transcoding_notebooks/data/mannwhitney_conf_int/posval.csv"
)
negval.to_csv(
    "/workspaces/pyplier/transcoding_notebooks/data/mannwhitney_conf_int/negval.csv"
)

In [97]:
x.to_csv("/workspaces/pyplier/transcoding_notebooks/data/mannwhitney_conf_int/x.csv")
y.to_csv("/workspaces/pyplier/transcoding_notebooks/data/mannwhitney_conf_int/y.csv")

In [86]:
rank_compare_2indep(posval, negval, use_t=True)

<class 'statsmodels.stats.nonparametric.RankCompareResult'>
statistic = 4.224814495022301
pvalue = 9.332657102238729e-05
s1 = 2668583.3266946194
s2 = 145.87683691494183
var1 = 0.08455071771589778
var2 = 0.05002635010800474
var = 8.931464069694657
var_prob = 0.001574658686476491
nobs1 = 54
nobs2 = 5618
nobs = 5672
mean1 = 3778.3518518518517
mean2 = 2827.4469562121753
prob1 = 0.6676489590337935
prob2 = 0.33235104096620993
somersd1 = 0.335297918067587
somersd2 = -0.33529791806758014
df = 53.60453416649346
use_t = True
tuple = (4.224814495022301, 9.332657102238729e-05)

In [ ]:
# conf_int_low, conf_int_high = mannwhitneyu_conf_int(posval, negval)

In [17]:
# def AUC(labels: pd.Series, values: pd.Series) -> Dict[str, float]:

IndexError: list index out of range

In [23]:
i

'LV2'

In [24]:
j

Index(['IRIS_Monocyte-Day0', 'IRIS_Neutrophil-Resting', 'DMAP_MONO1',
       'PID_IL6_7PATHWAY'],
      dtype='object')

In [25]:
iiheldout

Index(['GAS6', 'MMP14', 'MARCKSL1', 'SPARC', 'CTSD', 'EPAS1', 'PALLD', 'PHC2',
       'LGALS3BP', 'SERPING1',
       ...
       'LDHA', 'LDHB', 'ACAP1', 'ACAP2', 'ACAP3', 'CFL2', 'CFL1', 'SELL',
       'GNGT2', 'SERPINH1'],
      dtype='object', name='gene', length=5785)

In [22]:
for i in tqdm(ii):
    iipath = plierRes.U.loc[(plierRes.U.loc[:, i] > 0), i].index
    if len(iipath) > 1:
        for j in tqdm(iipath):
            iiheldout = (
                pd.concat([priorMat.loc[:, j], priorMatcv.loc[:, j]], axis=1)
                .apply(
                    lambda x: True
                    if (x[0] == 0) or ((x[0] > 0) and (x[1] == 0))
                    else np.nan,  # use np.nan instead of False so that we can drop entries in the chain
                    axis=1,
                )
                .dropna()
                .index
            )
            aucres = AUC(priorMat.loc[iiheldout, j], plierRes.Z.loc[iiheldout, i])
            out_dict[j] = {
                "pathway": j,
                "LV index": i,
                "AUC": aucres["auc"],
                "p-value": aucres["pval"],
            }
            Uauc.loc[j, i] = aucres["auc"]
            Up.loc[j, i] = aucres["pval"]

        else:
            j = iipath
            iiheldout = (
                pd.concat([priorMat.loc[:, j], priorMatcv.loc[:, j]], axis=1)
                .apply(
                    lambda x: True
                    if (x[0] == 0) or ((x[0] > 0) and (x[1] == 0))
                    else np.nan,
                    axis=1,
                )
                .dropna()
                .index
            )
            aucres = AUC(priorMat.loc[iiheldout, j], plierRes.Z.loc[iiheldout, i])
            print(f"j: {j}")
            print(f"i: {i}")
            if isinstance(j, Iterable):
                for _ in j:
                    out_dict[_] = {
                        "pathway": _,
                        "LV index": i,
                        "AUC": aucres["auc"],
                        "p-value": aucres["pval"],
                    }
                    Uauc.loc[_, i] = aucres["auc"]
                    Up.loc[_, i] = aucres["pval"]
            elif isinstance(j, str):
                out_dict[j] = {
                    "pathway": j,
                    "LV index": i,
                    "AUC": aucres["auc"],
                    "p-value": aucres["pval"],
                }
                Uauc.loc[j, i] = aucres["auc"]
                Up.loc[j, i] = aucres["pval"]

  5%|4         | 1/21 [00:00<00:19,  1.03it/s]


TypeError: type of argument "labels" must be pandas.core.series.Series; got pandas.core.frame.DataFrame instead

In [19]:
out = pd.DataFrame.from_dict(out_dict, orient="index")
_, fdr, *_ = multipletests(out.loc[:, "p-value"], method="fdr_bh")
out.loc[:, "fdr"] = fdr
res = {"Uauc": Uauc, "Upval": Up, "summary": out}

In [20]:
out.head()

,pathway,LV index,AUC,p-value,fdr
IRIS_Monocyte-Day0,IRIS_Monocyte-Day0,LV5,0.5,0.500001,0.500001
IRIS_Neutrophil-Resting,IRIS_Neutrophil-Resting,LV18,0.5,0.500001,0.500001
DMAP_MONO1,DMAP_MONO1,LV5,0.5,0.500001,0.500001
PID_IL6_7PATHWAY,PID_IL6_7PATHWAY,LV2,0.5,0.500001,0.500001
REACTOME_PROCESSING_OF_CAPPED_INTRON_CONTAINING_PRE_MRNA,REACTOME_PROCESSING_OF_CAPPED_INTRON_CONTAININ...,LV30,0.5,0.500001,0.500001


In [21]:
out[out["p-value"] < 0.5]

,pathway,LV index,AUC,p-value,fdr


In [22]:
out_dict

{'IRIS_Monocyte-Day0': {'pathway': 'IRIS_Monocyte-Day0', 'LV index': 'LV5', 'AUC': 0.5, 'p-value': 0.5000011745723949}, 'IRIS_Neutrophil-Resting': {'pathway': 'IRIS_Neutrophil-Resting', 'LV index': 'LV18', 'AUC': 0.5, 'p-value': 0.5000011766738179}, 'DMAP_MONO1': {'pathway': 'DMAP_MONO1', 'LV index': 'LV5', 'AUC': 0.5, 'p-value': 0.5000011745723949}, 'PID_IL6_7PATHWAY': {'pathway': 'PID_IL6_7PATHWAY', 'LV index': 'LV2', 'AUC': 0.5, 'p-value': 0.500001265677214}, 'REACTOME_PROCESSING_OF_CAPPED_INTRON_CONTAINING_PRE_MRNA': {'pathway': 'REACTOME_PROCESSING_OF_CAPPED_INTRON_CONTAINING_PRE_MRNA', 'LV index': 'LV30', 'AUC': 0.5, 'p-value': 0.5000011329414156}, 'MIPS_SPLICEOSOME': {'pathway': 'MIPS_SPLICEOSOME', 'LV index': 'LV30', 'AUC': 0.5, 'p-value': 0.5000011329414156}, 'REACTOME_GENERIC_TRANSCRIPTION_PATHWAY': {'pathway': 'REACTOME_GENERIC_TRANSCRIPTION_PATHWAY', 'LV index': 'LV26', 'AUC': 0.5, 'p-value': 0.5000011627386589}, 'REACTOME_SIGNALING_BY_RHO_GTPASES': {'pathway': 'REACTOME_SI

In [52]:
Uauc

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,LV11,LV12,LV13,LV14,LV15,LV16,LV17,LV18,LV19,LV20,LV21,LV22,LV23,LV24,LV25,LV26,LV27,LV28,LV29,LV30
IRIS_Bcell-Memory_IgG_IgA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-Memory_IgM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-naive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-N0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-Th1-restimulated12hour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_IL4_2PATHWAY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PID_BCR_5PATHWAY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PID_TELOMERASEPATHWAY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
data_dir = Path("/workspaces/pyplier/")
residual = pd.read_csv(data_dir / "residual.csv.gz", index_col=0)
B = pd.read_csv(data_dir / "B.csv.gz", index_col=0)
Z = pd.read_csv(data_dir / "Z.csv.gz", index_col=0)
U = pd.read_csv(data_dir / "U.csv.gz", index_col=0)
C = pd.read_csv(data_dir / "C.csv.gz", index_col=0)
L1 = 18.43058
L2 = 36.86117
L3 = 0.0004307425
heldOutGenes = pd.read_csv(data_dir / "heldoutgenes.csv.gz", index_col=0)

In [62]:
heldOutGenes.groupby("pathway")["gene"].apply(lambda x: [_ for _ in x]).to_dict()

{'BIOCARTA_BCR_PATHWAY': ['ELK1', 'PPP3CA', 'MAP3K1', 'RAC1', 'PPP3CC', 'JUN'], 'BIOCARTA_BIOPEPTIDES_PATHWAY': ['FYN', 'SHC1', 'GNA11', 'STAT4', 'MAPT', 'GNAI1', 'PLCG1'], 'BIOCARTA_CARM_ER_PATHWAY': ['EP300', 'PELP1', 'GTF2E1', 'GTF2A1', 'BRCA1', 'HDAC11'], 'BIOCARTA_CHREBP2_PATHWAY': ['PRKAR2B', 'PPP2R5A', 'PRKAR1A', 'PRKAB1', 'MLX', 'FASN'], 'BIOCARTA_DEATH_PATHWAY': ['BID', 'FADD', 'CYCS', 'CASP7', 'APAF1', 'TNFSF12'], 'BIOCARTA_EGF_PATHWAY': ['STAT1', 'STAT5B', 'GRB2', 'STAT4', 'PRKCB', 'MAP3K1'], 'BIOCARTA_FAS_PATHWAY': ['MAP2K4', 'PARP1', 'DFFA', 'RIPK2', 'FAF1'], 'BIOCARTA_FCER1_PATHWAY': ['LYN', 'SHC1', 'MAP2K1', 'FCER1A', 'MAPK8', 'VAV1', 'MAP2K7'], 'BIOCARTA_FMLP_PATHWAY': ['CALM3', 'NFKBIA', 'CALM1', 'RELA', 'HRAS', 'GNA15'], 'BIOCARTA_GPCR_PATHWAY': ['NFATC2', 'PPP3CA', 'RPS6KA3', 'JUN', 'PPP3CB', 'PLCG1'], 'BIOCARTA_HIVNEF_PATHWAY': ['RASA1', 'CRADD', 'PTK2', 'ACTG1', 'PAK2', 'NUMA1', 'XIAP', 'TRADD', 'DFFB', 'CASP7', 'PARP1'], 'BIOCARTA_IL2RB_PATHWAY': ['HRAS', 'NMI', '

In [77]:
Z.shape

(5892, 30)

In [81]:
plierRes = PLIERResults(
    residual=residual,
    B=B,
    Z=Z,
    U=U,
    C=C,
    L1=L1,
    L2=L2,
    L3=L3,
    heldOutGenes=heldOutGenes.groupby("pathway")["gene"]
    .apply(lambda x: [_ for _ in x])
    .to_dict(),
)

In [93]:
plierRes.to_disk("/workspaces/pyplier/tests/data/crossVal/plierRes.json.gz")

True

In [24]:
i = "LV26"
j = pd.Index(["DMAP_DENDA1", "REACTOME_GENERIC_TRANSCRIPTION_PATHWAY"], dtype="object")

In [25]:
iipath = plierRes.U.loc[(plierRes.U.loc[:, i] > 0), i].index

In [26]:
iipath

Index(['DMAP_DENDA1', 'REACTOME_GENERIC_TRANSCRIPTION_PATHWAY'], dtype='object')

In [27]:
for j in iipath:
    print(j)

DMAP_DENDA1
REACTOME_GENERIC_TRANSCRIPTION_PATHWAY


In [28]:
j = "DMAP_DENDA1"

In [33]:
pd.concat([priorMat.loc[:, j], priorMatcv.loc[:, j]], axis=1).dropna().index

Index(['GAS6', 'MMP14', 'MARCKSL1', 'SPARC', 'CTSD', 'EPAS1', 'PALLD', 'PHC2',
       'LGALS3BP', 'SERPING1',
       ...
       'LDHA', 'LDHB', 'ACAP1', 'ACAP2', 'ACAP3', 'CFL2', 'CFL1', 'SELL',
       'GNGT2', 'SERPINH1'],
      dtype='object', name='gene', length=5892)

In [18]:
for j in tqdm(iipath):
    iiheldout = (
        pd.concat([priorMat.loc[:, j], priorMatcv.loc[:, j]], axis=1)
        .apply(
            lambda x: True
            if (x[0] == 0) or ((x[0] > 0) and (x[1] == 0))
            else np.nan,  # use np.nan instead of False so that we can drop entries in the chain
            axis=1,
        )
        .dropna()
        .index
    )
    aucres = AUC(priorMat.loc[iiheldout, j], plierRes.Z.loc[iiheldout, i])
    out_dict[j] = {
        "pathway": j,
        "LV index": i,
        "AUC": aucres["auc"],
        "p-value": aucres["pval"],
    }
    Uauc.loc[j, i] = aucres["auc"]
    Up.loc[j, i] = aucres["pval"]

100%|##########| 4/4 [00:00<00:00,  5.49it/s]


j: Index(['IRIS_Monocyte-Day0', 'IRIS_Neutrophil-Resting', 'DMAP_MONO1',
       'PID_IL6_7PATHWAY'],
      dtype='object')
i: LV2


100%|##########| 2/2 [00:00<00:00,  5.44it/s]


j: Index(['REACTOME_PROCESSING_OF_CAPPED_INTRON_CONTAINING_PRE_MRNA', 'MIPS_SPLICEOSOME'], dtype='object')
i: LV3


100%|##########| 2/2 [00:00<00:00,  5.48it/s]


...

100%|##########| 4/4 [00:00<00:00,  5.90it/s]


j: Index(['IRIS_DendriticCell-LPSstimulated',
       'REACTOME_CYTOKINE_SIGNALING_IN_IMMUNE_SYSTEM',
       'REACTOME_INTERFERON_SIGNALING',
       'REACTOME_INTERFERON_ALPHA_BETA_SIGNALING'],
      dtype='object')
i: LV27


100%|##########| 12/12 [00:01<00:00,  7.02it/s]


j: Index(['DMAP_TCELLA2', 'REACTOME_MITOCHONDRIAL_PROTEIN_IMPORT',
       'REACTOME_PEPTIDE_CHAIN_ELONGATION',
       'REACTOME_PROCESSING_OF_CAPPED_INTRON_CONTAINING_PRE_MRNA',
       'REACTOME_FORMATION_OF_THE_TERNARY_COMPLEX_AND_SUBSEQUENTLY_THE_43S_COMPLEX',
       'MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX',
       'REACTOME_RESPIRATORY_ELECTRON_TRANSPORT_ATP_SYNTHESIS_BY_CHEMIOSMOTIC_COUPLING_AND_HEAT_PRODUCTION_BY_UNCOUPLING_PROTEINS_',
       'KEGG_PARKINSONS_DISEASE', 'MIPS_SPLICEOSOME', 'REACTOME_TRANSLATION',
       'REACTOME_SRP_DEPENDENT_COTRANSLATIONAL_PROTEIN_TARGETING_TO_MEMBRANE',
       'MIPS_55S_RIBOSOME_MITOCHONDRIAL'],
      dtype='object')
i: LV30


100%|##########| 21/21 [06:09<00:00, 17.61s/it]
